In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from dtaidistance import dtw
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.metrics import davies_bouldin_score, silhouette_score
from scipy.spatial.distance import squareform
from sklearn_extra.cluster import KMedoids
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
df = pd.read_csv("/Users/kessorchao/Desktop/Comp. tools for DS/02807_Project/output/data/discharge_tables/discharge_table_2001_2022.csv", index_col=0, parse_dates=True)
df

### Viusalize the DTW of two time series to get the idea

### Resample time and length 

In [ ]:
# The fewest data points a time series has is 276
length = 500
resampled_dict = {}

for col in df.columns:
    s = df[col].dropna()

    # # skip series that do not have enough data
    # if len(s) < length:
    #     continue

    # resample to fixed length
    y_new = np.interp(
        np.linspace(0, 1, length),
        np.linspace(0, 1, len(s)),
        s.values
    )
    
    resampled_dict[col] = pd.Series(y_new, index=np.linspace(0, 1, length), name=col)

df_resampled = pd.concat(resampled_dict, axis=1)


In [ ]:
df_resampled

In [ ]:
# # Normalise the data as we are interested in the shape not the volumne
# data_scaled = (df_resampled - df_resampled.mean()) / df_resampled.std()

# # Transpose the dataframe
# data_T = data_scaled.T.values.copy()  

data_T = df_resampled.T.values
data_scaled = (data_T - data_T.mean(axis=1, keepdims=True)) / data_T.std(axis=1, keepdims=True)


### Compute the DTW distance

In [ ]:
# this will be used for clustering
distance_matrix = dtw.distance_matrix_fast(data_scaled, compact=True, parallel=True)

In [ ]:
distance_matrix = squareform(distance_matrix)

# convert to dataframe
distance_df = pd.DataFrame(distance_matrix, index=df_resampled.columns, columns=df_resampled.columns)
distance_df

#### Distance matrix

In [ ]:
# The computed matrix is large so we select a subset of series to look at
indices = np.random.choice(df_resampled.shape[1], size=100, replace=False)
series_names = df_resampled.columns[indices]


In [ ]:
subset = distance_df.loc[series_names, series_names]
num = len(subset)

plt.figure(figsize=(50,50))
sns.heatmap(subset, annot=True, cmap="viridis")
plt.title(f"DTW distance between {num} random series")
plt.show()

# Interpretation: Dark areas = small DTW distance | Bright areas = large DTW distance

# Clustering: K-Means - DBSCAN - Multivariate Clustering - (Hierarchical Clustering)

## kmedoids

### Evaluating the K's

In [ ]:
# Data
series_names = distance_df.index
distance_matrix = distance_df.values

k_range = range(2,11)

km_silhouette_scores = []
km_davies_index = []
km_elbow = []
km_assignments = {}

for k in k_range:
    model = KMedoids(n_clusters=k, metric="precomputed", random_state=42)
    labels = model.fit_predict(distance_matrix)
    
    # Store silhouette score
    sil = silhouette_score(distance_matrix, labels, metric="precomputed")
    km_silhouette_scores.append(sil)

    dbi = davies_bouldin_score(distance_matrix, labels)
    km_davies_index.append(dbi)
    
    # Compute WSS-like distortion (sum of distances to medoid)
    distortion = 0
    for cluster_id in range(k):
        members = np.where(labels == cluster_id)[0]
        medoid = model.medoid_indices_[cluster_id]
        distortion += distance_matrix[medoid, members].sum()
    
    km_elbow.append(distortion)

    # Save assignments for later use
    km_assignments[k] = pd.Series(labels, index=series_names, name=f"k={k}")


In [ ]:
# Plot elbow method (inertia)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot elbow method (inertia)
axes[0].plot(k_range, km_elbow, marker='o')
axes[0].set_title('Elbow Method - Inertia vs. Number of Clusters')
axes[0].set_xlabel('Number of Clusters')
axes[0].set_ylabel('Inertia')
axes[0].grid(True)

# Plot silhouette score
axes[1].plot(k_range, km_silhouette_scores, marker='o')
axes[1].set_title('Silhouette Score vs. Number of Clusters (higher is better)')
axes[1].set_xlabel('Number of Clusters')
axes[1].set_ylabel('Silhouette Score')
axes[1].grid(True)

# Plot Davies–Bouldin Index
axes[2].plot(k_range, km_davies_index, marker='o')
axes[2].set_title('Davies–Bouldin Index vs. Number of Clusters (lower is better)')
axes[2].set_xlabel('Number of Clusters')
axes[2].set_ylabel('Davies–Bouldin Index')
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
assignments_df = pd.concat(km_assignments, axis=1)
assignments_df

In [ ]:
# from the assignments_df count the number of labels for each n_clusters
label_counts = {
    col: assignments_df[col].value_counts().sort_index()
    for col in assignments_df.columns
}

label_counts_df = pd.DataFrame(label_counts).fillna(0).astype(int)
label_counts_df

In [ ]:
def plot_cluster(assignments_df, k, cluster_id):
    series_in_cluster = assignments_df.index[assignments_df[k] == cluster_id]
    
    plt.figure(figsize=(10,6))
    colors = cm.tab20(np.linspace(0, 1, len(series_in_cluster)))
    
    for color, series_name in zip(colors, series_in_cluster):
        plt.plot(df_resampled.index, df_resampled[series_name], color=color, linewidth=2) #, label=series_name)
    
    plt.title(f"Time Series in Cluster {cluster_id} (k={k})")
    plt.xlabel("Relative time (0→1)")
    plt.ylabel("Discharge")
    plt.legend(loc='upper right', fontsize=8)
    plt.show()

In [ ]:
plot_cluster(assignments_df, 10, 5)

### Save one k clustering into a csv file for visualization in another file

In [ ]:
# Choose the column you want to save
col_name = 10  # replace with your column name

# Select the column and reset the index to make catchment_id a regular column
output_df = assignments_df[[col_name]].reset_index()
output_df.rename(columns={'index': 'catchment_id', 10: 'cluster'}, inplace=True)

# Save to CSV
output_df.to_csv('kmedoids_labels.csv', index=False)


## DBSCAN

In [ ]:
eps_values = range(1,20)
min_samples_values = [2]

dbscan_results = []

for eps in eps_values:
    for min_s in min_samples_values:
        model = DBSCAN(metric='precomputed', eps=eps, min_samples=min_s)
        labels = model.fit_predict(distance_matrix)
        
        # Count number of clusters (excluding noise)
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # Compute silhouette score (only if there is more than 1 cluster)
        if n_clusters > 1:
            dbscan_silscore = silhouette_score(distance_matrix, labels)
            dbscan_dbi = davies_bouldin_score(distance_matrix, labels)
        else:
            dbscan_silscore = np.nan 
            dbscan_dbi = np.nan 
        
        
        dbscan_results.append({
            "eps": eps,
            "min_samples": min_s,
            "n_clusters": n_clusters,
            "noise points": n_noise,
            "silhouette_score": dbscan_silscore,
            "davies_index": dbscan_dbi
        })

df_dbscan = pd.DataFrame(dbscan_results)
display(df_dbscan)
df_dbscan.plot(x="eps", y=["silhouette_score", "davies_index"], kind="line", figsize=(12, 6))

## Evaluating the optimal eps

In [ ]:
# Remove rows where silhouette or DBI are NaN
df_valid = df_dbscan.dropna(subset=['silhouette_score', 'davies_index'])

# Optimal eps by highest silhouette score
best_sil_row = df_valid.loc[df_valid['silhouette_score'].idxmax()]
best_sil_eps = best_sil_row['eps']
print(f"Best eps by silhouette score: {best_sil_eps} (score={best_sil_row['silhouette_score']:.3f})")

# Optimal eps by lowest Davies–Bouldin Index
best_dbi_row = df_valid.loc[df_valid['davies_index'].idxmin()]
best_dbi_eps = best_dbi_row['eps']
print(f"Best eps by Davies–Bouldin Index: {best_dbi_eps} (DBI={best_dbi_row['davies_index']:.3f})")


In [ ]:
# DBSCAN parameters
eps = 9
min_samples = 2

dbscan = DBSCAN(metric='precomputed', eps=eps, min_samples=min_samples)
dbscan.fit(distance_matrix)

db_labels = dbscan.labels_  


In [ ]:
cluster_dbscan = pd.DataFrame({
    'series': df_resampled.columns,
    'cluster': db_labels
})

cluster_count_dbscan = pd.Series(db_labels).value_counts().sort_index()
cluster_count_dbscan

In [ ]:
dbscan_assignment = pd.DataFrame({
    'series': df_resampled.columns,
    'cluster': db_labels
})

dbscan_assignment


# Select the column and reset the index to make catchment_id a regular column
output_df = dbscan_assignment
output_df.rename(columns={'series': 'catchment_id'}, inplace=True)

# Save to CSV
output_df.to_csv('dbscan_labels.csv', index=False)


### Plot DBSCAN using t-SNE

In [ ]:
# t-SNE projection
tsne = TSNE(n_components=2, metric='precomputed', init='random', random_state=42)
X_2d = tsne.fit_transform(distance_matrix)

# Plot
plt.figure(figsize=(8,6))
unique_labels = set(db_labels)
# colors = [plt.cm.tab20(i) for i in range(len(unique_labels))]
colors = (
    list(plt.cm.tab20.colors) +
    list(plt.cm.tab20b.colors) +
    list(plt.cm.tab20c.colors)
)

for k, col in zip(unique_labels, colors):
    class_member_mask = (db_labels == k)
    xy = X_2d[class_member_mask]
    plt.scatter(xy[:, 0], xy[:, 1], c=[col], label=f'Cluster {k}', s=50)

plt.title('DBSCAN Clustering (t-SNE projection)')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend()
plt.show()

In [ ]:
# Plot each cluster to understand the scatterplot

# Plot each cluster except noise (-1)
for cluster in np.unique(db_labels):
    if cluster == -1:
        continue  # skip noise
    plt.figure(figsize=(12,4))
    
    # Get indices of time series in this cluster
    indices = np.where(db_labels == cluster)[0]
    
    for idx in indices:
        name = series_names[idx]
        plt.plot(df_resampled[name], alpha=0.5) #, label=name)
    
    plt.title(f'Cluster {cluster}')
    plt.legend(loc='upper right', fontsize=8)
    plt.show()


#### Evaluate

In [ ]:
X = df_resampled.T.values  # shape: (570 series × 276 points)
dbscan_labels = dbscan.labels_

dbscan_db_index = davies_bouldin_score(X, dbscan_labels)
dbscan_silhouette_avg = silhouette_score(X, dbscan_labels)
print("Davies-Bouldin Index:", dbscan_db_index)
print(f"Silhouette Score: {dbscan_silhouette_avg}")

### Save dbscan labels into a csv file for visualization in another file

In [ ]:
# Choose the column you want to save
col_name = 10 

# Select the column and reset the index to make catchment_id a regular column
output_df = assignments_df[[col_name]].reset_index()
output_df.rename(columns={'index': 'catchment_id', 10: 'cluster'}, inplace=True)

# Save to CSV
output_df.to_csv('dbscan_labels.csv', index=False)


## Multivariate clustering

### PLAN
##### DTW
##### Clustering: KMeans/kmedoids (DONE), DBSCAN (DONE), Multivariate clustering, (Hierarchical clustering)
##### Evaluation: Davies Bouldin Index, Shilhouette Score, (Calinski-Harabasz Index)

### Visualization
##### DTW - e.g. 2D
##### Time series graph
##### Table metrics